## Disclaimer

If you're reading this, I'll assume you've installed and tried running the basic flows in aceflow. This notebook illustrates some of the more "interesting" tricks to parameterize ACE potentials. These tricks aren't usually of much use, as the basics notebook covers everything needed to fit a good ACE potential. The contents of this notebook should **only** be tried if the default ACE parametrization doesn't work for your system of interest. Everthing here is once again a wrapper around pacemaker, and a full understanding of what's happening behind the hood can be inferred from the source code of pyace/python-ace. 

## 1. Changing Basis Function specification

Since ACE is in essense a fit over precomputed basis functions, changing the type (for both radial and angular) of functions is another knob to better fit a potential for your system. By default, the potential is built with the configuration given in aceflow/reference_objects/input_template.yaml. This template has been adapted from existing ACE literature, and should work for most cases. However, if you wish to tweak the parameters:

In [ ]:
from aceflow.utils.config import TrainConfig
from aceflow.reference_objects.BBasis_classes import BBasisBonds, BBasisEmbedding, UnaryBBasisOrder, BinaryBBasisOrder

bonds = BBasisBonds()
embedding = BBasisEmbedding()
unary_order = UnaryBBasisOrder()
binary_order = BinaryBBasisOrder()

bbasis_config = {
    "bonds": bonds,
    "embedding": embedding,
    "unary_order": unary_order,
    "binary_order": binary_order
}

train_config = TrainConfig(bbasis = bbasis_config)

The number and type of configuration for each part of the interaction (unary, binary, bonds, etc) can be set when instatiating the classes, such as UnaryBBasisOrder(). Similarly the order and coefficients of the embedding function (defaults to a complex Finnis-Sinclair type potential) can be changed through the BBasisEmbedding object. A smaller potential can be built by only passing bonds, embedding, unary and binary for a large system, which might be easier to train and pick up from (see next section).

### 2. Training specific interactions only

Another utility of ACE is to be able to only train specific parts of the potential, for example, train only binary interactions in a ternary system, while freezing the rest, to improve the model's performance in cases where only the binary interaction should be used. This can be done by specifying bbasis_train_orders in TrainConfig:

In [ ]:
train_config = TrainConfig(bbasis_train_orders=[0,1]) #trains unary and binary orders coefficients only
train_config = TrainConfig(bbasis_train_orders=[0,1,2]) #trains unary, binary and ternary orders coefficients only
train_config = TrainConfig(bbasis_train_orders=[2]) #trains ternary order coefficients only

This way, for large or tricky to parametrize systems, potentials can be built up in a systematic manner, training the unary and binaries first, then the ternaries only, and then a final fine tuning run using all the parameters. The HeirarchicalACEMaker flow does precisely this. (Note: this flow is still WIP).

In [ ]:
from aceflow.flows.trainer import HeirarchicalACEMaker
from aceflow.utils.config import HeirarchicalFitConfig

hfit_config = HeirarchicalFitConfig()
maker = HeirarchicalACEMaker(train_config, hfit_config)
flow = maker.make(data)